In [ ]:
#!git --version
#!git clone https://github.com/evdokiazorina/homeworks.git

In [ ]:
import requests

In [ ]:
### 1. Получить список функций белка P04637 (p53)
url = "https://www.ebi.ac.uk/proteins/api/proteins/P04637"
response = requests.get(url, headers={"Accept": "application/json"})

data = response.json()
for feature in data["features"]:
    feature_type = feature.get("type", "")
    description = feature.get("description", "")
    print(f"{feature_type}: {description}")

In [ ]:
###2. Получите последовательности белков P05067 (APP) и P10636 (Tau). Сравните их длины. Какой белок длиннее и на сколько аминокислот?
proteins = ["P05067", "P10636"]

sequences = {}

for p in proteins:
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{p}"
    response = requests.get(url, headers={"Accept": "application/json"})
    data = response.json()
    sequence = data["sequence"]["sequence"]
    sequences[p] = sequence

In [ ]:
print(f'Длина P05067:{len(sequences["P05067"])}, длина P10636:{len(sequences["P10636"])}')

Вывод: P05067 длиннее на 12 аминокислот

In [ ]:
#3 Получите данные о 5 различных белках (на ваш выбор) через API, используя их accession IDs. 
#Сохраните их entryName и длину последовательности в таблицу (data frame в R/Pandas)

In [ ]:
import pandas as pd

In [ ]:
accessions = ["P68871", "P00734", "P01308", "P68133", "Q9Y6K9"]
result = [] #наш будущий датафрейм

for accession in accessions:
    response = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}")
    data = response.json()
    entry_name = data.get('id', 'N/A')
    sequence_length = len(data['sequence']['sequence'])
    
    result.append({
        'accession': accession,
        'entryName': entry_name,
        'sequenceLength': sequence_length
    })

In [ ]:
df = pd.DataFrame(result)
df

In [ ]:
#4
#(Python/R): Используя API PubMed, найдите 10 самых последних (сортировка по дате) 
#статей по запросу "COVID-19 variants". Выведите их ID и заголовки.

In [ ]:
search_response = requests.get( #ищем статьи
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={
        'db': 'pubmed',
        'term': '"COVID-19 variants"',
        'retmode': 'json',
        'retmax': 10,
        'sort': 'pub+date'
    }
)
search_data = search_response.json()
ids = search_data['esearchresult']['idlist']

fetch_response = requests.get( #получаем заголовки
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",
    params={
        'db': 'pubmed',
        'id': ','.join(ids),
        'retmode': 'json'
    }
)
articles_data = fetch_response.json()

for pubmed_id in ids:
    article = articles_data['result'][pubmed_id]
    print(f"{pubmed_id}: {article.get('title', 'No title')}")

In [ ]:
# 5 (Python): Напишите функцию get_protein_name(accession), которая принимает на вход accession ID белка и возвращает его название (entryName). Обработайте случай, если белок не найден.
def get_protein_name(accession):
    response = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}")
    if response.status_code == 200:
        data = response.json()
        return data.get('protein', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'Name not found')
    else:
        return f"Error: Protein not found"

In [ ]:
print(get_protein_name("P04639")) #пример работы

In [ ]:
#6 (Python): Прочитайте из файла accessions.txt (нужно создать) список из 3-5 accession IDs.
#Для каждого ID получите из Proteins API и запишите в новый файл его название и организм-источник.

In [ ]:
with open("accessions.txt", "w") as f: #пзаполяем файлик
    f.write("P04637\nP05067\nP10636\nP68871\nP00734\n")

with open('accessions.txt', 'r') as file: #чтение айди
    accessions = [line.strip() for line in file if line.strip()]

with open('protein_info.txt', 'w') as output_file: #запись в файл
    output_file.write("Accession\tEntryName\tOrganism\n")
    
    for accession in accessions:
        response = requests.get(f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}")
        if response.status_code == 200:
            data = response.json()
            entry_name = data.get('id', 'N/A')
            organism = data.get('organism', {}).get('names', [{}])[0].get('value', 'N/A')
            output_file.write(f"{accession}\t{entry_name}\t{organism}\n")

print("Информация сохранена в protein_info.txt")

In [ ]:
# 7
#(Python/R): Для найденных в задании 4 статей получите более подробную информацию, 
#используя Eutils API и ID статей (конечная точка efetch.fcgi). Извлеките имена авторов для каждой статьи.
import xml.etree.ElementTree as ET
search_response = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi",
    params={
        'db': 'pubmed',
        'term': '"COVID-19 variants"',
        'retmode': 'json',
        'retmax': 4
    }
)
search_data = search_response.json()
article_ids = search_data['esearchresult']['idlist']
fetch_response = requests.get(
    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
    params={
        'db': 'pubmed',
        'id': ','.join(article_ids),
        'retmode': 'xml'
    }
)

root = ET.fromstring(fetch_response.content)
for article in root.findall('.//PubmedArticle'):
    pmid = article.find('.//PMID').text
    authors = article.findall('.//Author')
    author_names = []
    for author in authors:
        last_name = author.find('LastName')
        fore_name = author.find('ForeName')
        if last_name is not None and fore_name is not None:
            author_names.append(f"{fore_name.text} {last_name.text}")
    
    print(f"ID: {pmid}")
    print(f"Авторы: {', '.join(author_names)}")

In [ ]:
### 8 (Bash): Напишите команду curl, которая получает данные о белке, выполняет поиск по слову "phosphorylation" 
#(либо другое на ваш выбор) в ответе и выводит строки, содержащие это слово.
!curl -s "https://www.ebi.ac.uk/proteins/api/proteins/P04637" | grep -i "phosphorylation"

In [ ]:
###9. Белок Q8WZ42 (Titin):  получить последовательность и разбить на строки по 80 символов

In [ ]:
!curl -s "https://www.ebi.ac.uk/proteins/api/proteins/Q8WZ42" | grep -o '"sequence":"[^"]*"' | cut -d'"' -f4 | fold -w 80

In [ ]:
## 10
library(httr)
library(jsonlite)

accessions <- c("P04637", "P05067", "P10636", "P03372", "Q8WZ42", "P01375", "P61586", "P62158", "P0DTD1", "P68871")
lengths <- c()
names <- c()

for (accession in accessions) {
  response <- GET(paste0("https://www.ebi.ac.uk/proteins/api/proteins/", accession))
  data <- fromJSON(content(response, "text"))
  lengths <- c(lengths, nchar(data$sequence$sequence))
  names <- c(names, data$id)
}

barplot(lengths, names.arg = names, las = 2, main = "Длина белков", ylab = "Аминокислоты")

In [ ]:
##11 не получилось

In [ ]:
#12. (Python): Ваш коллега написал код, который в цикле делает 100 запросов к API (по 1 белку). 
#Перепишите код, чтобы сделать всего 1 запрос, получающий данные сразу по 100 белкам (используйте соответствующий параметр в Proteins API)

accessions = ["P04637", "P05067", "P10636", "P03372", "Q8WZ42"] #пример списка

acc_str = ",".join(accessions)
url = "https://www.ebi.ac.uk/proteins/api/proteins"
params = {"accession": acc_str}
r = requests.get(url, params=params, headers={"Accept":"application/json"})
data = r.json()
data # data теперь содержит список результатов для всех accession


In [ ]:
#13
library(httr)
acc <- "P05067"
res <- GET(paste0("https://www.ebi.ac.uk/proteins/api/proteins/", acc), add_headers(Accept="text/x-fasta"))
if (status_code(res)==200) {
  writeBin(content(res, "raw"), paste0(acc, ".fasta"))
  cat("Saved", paste0(acc, ".fasta"), "\n")
} else {
  cat("Error", status_code(res), "\n")
}


In [ ]:
#14
response = requests.get("https://www.ebi.ac.uk/proteins/api/nonexistent_endpoint") #endpoint,который не существует

if response.status_code == 404:
    print("Ошибка 404: Белок не найден")
elif response.status_code == 200:
    print("Запрос успешен")
else:
    print(f"Получен код ответа: {response.status_code}")

In [ ]:
#16
!curl -s "https://www.ebi.ac.uk/proteins/api/proteins/P04637" > protein_data.json #делаем файл

In [36]:
!jq -r '.accession' protein_data.json

P04637


In [48]:
#17
#информация
data1 = requests.get("https://www.ebi.ac.uk/proteins/api/proteins/P10000").json()
print("Название:", data1['protein']['recommendedName']['fullName']['value'])
print("Ген:", data1['gene'][0]['name']['value'])

#поиск статей
pubmed_data = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=APP[Gene]&retmode=json&retmax=5").json()
article_ids = pubmed_data['esearchresult']['idlist']
print("Последние статьи:", article_ids[:3])

Название: Pro-opiomelanocortin
Ген: pomc
Последние статьи: ['41196440', '41196411', '41195969']


In [58]:
%cd homeworks


print("=== Текущий статус ===")
!git status


!git checkout main
!git pull origin main

/Users/eyzorina/Учеба/homeworks
=== Текущий статус ===
On branch dev
Your branch is ahead of 'origin/dev' by 2 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean

=== Обновляем main ===
Switched to branch 'main'
Your branch is up to date with 'origin/main'.
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (1/1), 890 bytes | 445.00 KiB/s, done.
From https://github.com/evdokiazorina/homeworks
 * branch            main       -> FETCH_HEAD
   d9a7bbc..ee40221  main       -> origin/main
Updating d9a7bbc..ee40221
Fast-forward
 Homework4.R |  45 +++++++++++++++++++++++++
 Homework5.R | 107 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 homework3   |  67 ++++++++++++++++++++++---------------
 3 files changed, 192 insertions(+), 27 deletions(-)
 create mode 100644 Homework4.R
 create mode 100644 Homework5.R


In [56]:
!git checkout main
!git pull origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
